In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1399424712295831117,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1535151976301995835
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7619718294642143537
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2494062817374645499
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2, l1

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.4.3
tensorflow Version 2.3.0


In [6]:
batch_size = 32
img_height, img_width = 800, 600
input_shape = (img_height, img_width, 3)
epochs = 1000

In [8]:
print(os.listdir("../input/data-pig/"))

['train', 'test']


In [9]:
train_dir = '../input/data-pig/train/'
test_dir = '../input/data-pig/test/'

In [10]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [11]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
#     rotation_range=90,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.3,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='reflect',
    validation_split= 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 8061 images belonging to 4 classes.
Found 2012 images belonging to 4 classes.
Found 1776 images belonging to 4 classes.


In [12]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 8061
nb_validation_samples: 2012
nb_test_samples: 1776

predict_size_train: 252
predict_size_validation: 63
predict_size_test: 56

 num_classes: 4


In [13]:
os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "VGG19_descriptors"

In [14]:
vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"
xception_weights = "../input/full-keras-pretrained-no-top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [15]:
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
from keras.applications import MobileNet

In [16]:
input_tensor = Input(shape = input_shape)  

base_model1=InceptionV3(weights=inception_weights, include_top=False, pooling = "avg")

In [17]:
# base_model1.summary()

In [18]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [19]:
bottleneck_final_model = base_model1

In [21]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [22]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [23]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [24]:

# import tensorflow as tf
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))

# reset_keras_tf_session()

In [25]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [26]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        'hidden1':hp.choice('hidden1',[4096,2048,1024,512,256]),
#         'hidden2':hp.choice('hidden2',[2048,1024,512,256,128,]),
#         'hidden3':hp.choice('hidden3',[1024,512,256,128,64,32]),    
        'bias_reg': hp.choice('bias_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'act_reg': hp.choice('act_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'ker_reg': hp.choice('ker_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'activation_function':hp.choice('activation_function',["relu","elu","selu","softplus","tanh",])
        }

In [27]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
#     dropout_rate = 0.5
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(params["hidden1"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
#     model.add(Dropout(0.25))

#     model.add(Dense(params["hidden2"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    
    
#     model.add(Dense(params["hidden3"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    
    
    model.add(Dropout(0.5))
    

    model.add(Dense(num_classes, activation="softmax"))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [28]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [29]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:                                    
{'act_reg': 0.1, 'activation_function': 'elu', 'beta_1': 0.9, 'beta_2': 0.6, 'bias_reg': 0.0001, 'hidden1': 4096, 'ker_reg': 0.01, 'lr': 1e-06}
Epoch 1/1000                                           
252/252 - 1s - loss: 97.9455 - accuracy: 0.2044 - val_loss: 79.4913 - val_accuracy: 0.1794

Epoch 2/1000                                           
252/252 - 1s - loss: 67.6538 - accuracy: 0.2867 - val_loss: 58.8841 - val_accuracy: 0.3748

Epoch 3/1000                                           
252/252 - 1s - loss: 53.2571 - accuracy: 0.5042 - val_loss: 50.1960 - val_accuracy: 0.8489

Epoch 4/1000                                           
252/252 - 1s - loss: 47.4402 - accuracy: 0.7999 - val_loss: 47.0785 - val_accuracy: 0.8733

Epoch 5/1000                                           
252/252 - 1s - loss: 45.2588 - accuracy: 0.8963 - val_loss: 45.9017 - val_accuracy: 0.9145

Epoch 6/1000                                           
252/

Epoch 110/1000                                         
252/252 - 1s - loss: 30.0314 - accuracy: 0.9998 - val_loss: 31.7547 - val_accuracy: 0.9916

Epoch 111/1000                                         
252/252 - 1s - loss: 29.9539 - accuracy: 0.9999 - val_loss: 31.6702 - val_accuracy: 0.9916

Epoch 112/1000                                         
252/252 - 1s - loss: 29.8764 - accuracy: 0.9999 - val_loss: 31.5803 - val_accuracy: 0.9916

Epoch 113/1000                                         
252/252 - 1s - loss: 29.7978 - accuracy: 0.9998 - val_loss: 31.5129 - val_accuracy: 0.9916

Epoch 114/1000                                         
252/252 - 1s - loss: 29.7203 - accuracy: 0.9998 - val_loss: 31.4301 - val_accuracy: 0.9916

Epoch 115/1000                                         
252/252 - 1s - loss: 29.6430 - accuracy: 0.9999 - val_loss: 31.3569 - val_accuracy: 0.9916

Epoch 116/1000                                         
252/252 - 1s - loss: 29.5674 - accuracy: 0.9999 - val_lo

Epoch 220/1000                                         
252/252 - 1s - loss: 22.9907 - accuracy: 0.9999 - val_loss: 24.4759 - val_accuracy: 0.9940

Epoch 221/1000                                         
252/252 - 1s - loss: 22.9373 - accuracy: 1.0000 - val_loss: 24.4147 - val_accuracy: 0.9940

Epoch 222/1000                                         
252/252 - 1s - loss: 22.8856 - accuracy: 1.0000 - val_loss: 24.3634 - val_accuracy: 0.9940

Epoch 223/1000                                         
252/252 - 1s - loss: 22.8333 - accuracy: 1.0000 - val_loss: 24.2987 - val_accuracy: 0.9940

Epoch 224/1000                                         
252/252 - 1s - loss: 22.7813 - accuracy: 0.9999 - val_loss: 24.2500 - val_accuracy: 0.9935

Epoch 225/1000                                         
252/252 - 1s - loss: 22.7300 - accuracy: 0.9999 - val_loss: 24.1983 - val_accuracy: 0.9940

Epoch 226/1000                                         
252/252 - 1s - loss: 22.6785 - accuracy: 1.0000 - val_lo

Epoch 330/1000                                         
252/252 - 1s - loss: 18.0802 - accuracy: 1.0000 - val_loss: 19.3707 - val_accuracy: 0.9960

Epoch 331/1000                                         
252/252 - 1s - loss: 18.0430 - accuracy: 1.0000 - val_loss: 19.3374 - val_accuracy: 0.9950

Epoch 332/1000                                         
252/252 - 1s - loss: 18.0051 - accuracy: 0.9999 - val_loss: 19.2968 - val_accuracy: 0.9955

Epoch 333/1000                                         
252/252 - 1s - loss: 17.9678 - accuracy: 1.0000 - val_loss: 19.2608 - val_accuracy: 0.9960

Epoch 334/1000                                         
252/252 - 1s - loss: 17.9307 - accuracy: 1.0000 - val_loss: 19.2157 - val_accuracy: 0.9960

Epoch 335/1000                                         
252/252 - 1s - loss: 17.8925 - accuracy: 0.9999 - val_loss: 19.1778 - val_accuracy: 0.9960

Epoch 336/1000                                         
252/252 - 1s - loss: 17.8559 - accuracy: 0.9999 - val_lo

Epoch 440/1000                                         
252/252 - 1s - loss: 14.4837 - accuracy: 1.0000 - val_loss: 15.6225 - val_accuracy: 0.9965

Epoch 441/1000                                         
252/252 - 1s - loss: 14.4553 - accuracy: 1.0000 - val_loss: 15.5943 - val_accuracy: 0.9965

Epoch 442/1000                                         
252/252 - 1s - loss: 14.4280 - accuracy: 1.0000 - val_loss: 15.5665 - val_accuracy: 0.9970

Epoch 443/1000                                         
252/252 - 1s - loss: 14.3993 - accuracy: 1.0000 - val_loss: 15.5353 - val_accuracy: 0.9970

Epoch 444/1000                                         
252/252 - 1s - loss: 14.3712 - accuracy: 1.0000 - val_loss: 15.5085 - val_accuracy: 0.9965

Epoch 445/1000                                         
252/252 - 1s - loss: 14.3436 - accuracy: 1.0000 - val_loss: 15.4792 - val_accuracy: 0.9965

Epoch 446/1000                                         
252/252 - 1s - loss: 14.3162 - accuracy: 1.0000 - val_lo

...

Epoch 988/1000                                                                           
252/252 - 1s - loss: 1.3590 - accuracy: 0.3801 - val_loss: 1.3598 - val_accuracy: 0.3802 

Epoch 989/1000                                                                           
252/252 - 1s - loss: 1.3589 - accuracy: 0.3801 - val_loss: 1.3598 - val_accuracy: 0.3802 

Epoch 990/1000                                                                           
252/252 - 1s - loss: 1.3590 - accuracy: 0.3801 - val_loss: 1.3598 - val_accuracy: 0.3802 

Epoch 991/1000                                                                           
252/252 - 1s - loss: 1.3589 - accuracy: 0.3801 - val_loss: 1.3597 - val_accuracy: 0.3802 

Epoch 992/1000                                                                           
252/252 - 1s - loss: 1.3588 - accuracy: 0.3801 - val_loss: 1.3597 - val_accuracy: 0.3802 

Epoch 993/1000                                                                           
252/2

252/252 - 1s - loss: 1.5303 - accuracy: 0.2294 - val_loss: 1.5338 - val_accuracy: 0.2271  

Epoch 67/1000                                                                             
252/252 - 1s - loss: 1.5153 - accuracy: 0.2399 - val_loss: 1.5188 - val_accuracy: 0.2346  

Epoch 68/1000                                                                             
252/252 - 1s - loss: 1.5019 - accuracy: 0.2249 - val_loss: 1.5051 - val_accuracy: 0.2425  

Epoch 69/1000                                                                             
252/252 - 1s - loss: 1.4900 - accuracy: 0.2265 - val_loss: 1.4926 - val_accuracy: 0.2530  

Epoch 70/1000                                                                             
252/252 - 1s - loss: 1.4796 - accuracy: 0.2345 - val_loss: 1.4811 - val_accuracy: 0.2753  

Epoch 71/1000                                                                             
252/252 - 1s - loss: 1.4697 - accuracy: 0.2315 - val_loss: 1.4708 - val_accuracy: 0.3

252/252 - 1s - loss: 1.3863 - accuracy: 0.3964 - val_loss: 1.3867 - val_accuracy: 0.3882  

Epoch 156/1000                                                                            
252/252 - 1s - loss: 1.3863 - accuracy: 0.3985 - val_loss: 1.3867 - val_accuracy: 0.3887  

Epoch 157/1000                                                                            
252/252 - 1s - loss: 1.3863 - accuracy: 0.3978 - val_loss: 1.3867 - val_accuracy: 0.3882  

Epoch 158/1000                                                                            
252/252 - 1s - loss: 1.3863 - accuracy: 0.3977 - val_loss: 1.3867 - val_accuracy: 0.3882  

Epoch 159/1000                                                                            
252/252 - 1s - loss: 1.3863 - accuracy: 0.3990 - val_loss: 1.3867 - val_accuracy: 0.3882  

Epoch 160/1000                                                                            
252/252 - 1s - loss: 1.3863 - accuracy: 0.3987 - val_loss: 1.3867 - val_accuracy: 0.3

252/252 - 1s - loss: 1.3858 - accuracy: 0.4425 - val_loss: 1.3862 - val_accuracy: 0.3966  

Epoch 245/1000                                                                            
252/252 - 1s - loss: 1.3858 - accuracy: 0.4424 - val_loss: 1.3862 - val_accuracy: 0.3961  

Epoch 246/1000                                                                            
252/252 - 1s - loss: 1.3859 - accuracy: 0.4383 - val_loss: 1.3862 - val_accuracy: 0.3951  

Epoch 247/1000                                                                            
252/252 - 1s - loss: 1.3858 - accuracy: 0.4379 - val_loss: 1.3862 - val_accuracy: 0.3941  

Epoch 248/1000                                                                            
252/252 - 1s - loss: 1.3858 - accuracy: 0.4421 - val_loss: 1.3862 - val_accuracy: 0.3941  

Epoch 249/1000                                                                            
252/252 - 1s - loss: 1.3858 - accuracy: 0.4429 - val_loss: 1.3862 - val_accuracy: 0.3

KeyboardInterrupt: 

In [ ]:
print("Done!!!!!!!!!!!!!!!!!!!")